<a href="https://colab.research.google.com/github/MarceloAbreu90/Gemini-Maint-AI/blob/main/GeminiMaintAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai clear-screen

In [47]:
#Importação das Bibliotecas:
import requests
import google.generativeai as genai
import sys
import time
from google.colab import userdata
from clear_screen import clear

In [39]:
#Configuração da url do arquivo JSON máquinas
url_maquinas = "https://raw.githubusercontent.com/MarceloAbreu90/Gemini-Maint-AI/main/dados/maquinas.json"

In [48]:
#Configuração da Chave API:
api_kei = userdata.get('SECRET_KEY')
genai.configure(api_key=api_kei)

In [6]:
#Parâmetros de Segurança:
safety_settings={
    'HATE': 'block_only_high',
    'HARASSMENT': 'block_only_high',
    'SEXUAL' : 'block_only_high',
    'DANGEROUS' : 'block_only_high'
    }

In [34]:
#Passando System Instructions para a Gemini AI:
system_instructions = """
Você é um assistente de manutenção industrial chamado Gemini Maint AI, especializado em ajudar operadores de máquinas a resolver problemas, você pode ajudar com seu conhecimento e também consultando o manual da máquina que será disponibilizado via link, sempre cite as paginas do manual que contém as informações que você utilizar e envie no final da sua resposta o link do manual para que o usuário possa consultar caso necessário e avisos de execução de manutenção com segurança.
"""

In [8]:
#Configuração do Modelo:
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    safety_settings=safety_settings
)

In [40]:
#Realiza chamada do arquivo JSON
response = requests.get(url_maquinas)
data = response.json()

In [21]:
#Função menu principal:
def exibir_menu_principal():
    clear()
    print("Seja bem-vindo ao Gemini Maint AI, seu assistente de manutenção industrial!")
    print("\n*** MENU PRINCIPAL ***")
    print("\nPor favor, informe a máquina para obter ajuda:\n")

    while True:
        for i, maquina in enumerate(data):
            print(f"{i+1} - {maquina['nome']} - modelo: {maquina['modelo']}")

        print("\n0 - Sair")  # Adicionar opção 0

        escolha = input("\nDigite o número da máquina: ")

        try:
            escolha_int = int(escolha)
            if escolha_int == 0:
                print("Encerrando o Gemini Maint AI. Até logo!")
                sys.exit()  # Encerrar o script
            elif 1 <= escolha_int <= len(data):
                maquina_selecionada = data[escolha_int - 1]
                exibir_defeitos(maquina_selecionada)
                break  # Sai do loop se a escolha for válida
            else:
                print("Escolha inválida. Por favor, digite um número válido.")
        except ValueError:
            print("Escolha inválida. Por favor, digite um número.")

In [31]:
#Função menu defeitos:
def exibir_defeitos(maquina):
    clear()
    print("\n\n----------------------------------------\n\n")
    print("*** TIPOS DE DEFEITOS ***")
    print(f"\nMáquina: {maquina['nome']}")
    print(f"Modelo: {maquina['modelo']}")
    print(f"Fabricante: {maquina['fabricante']}")
    print("\nPor favor, informe o defeito:\n")

    while True:
        for i, defeito in enumerate(maquina['defeitos']):
            print(f"{i+1} - {defeito['descricao']}")

        print("\n0 - Voltar ao menu principal")  # Adicionar opção 0

        escolha_defeito = input("\nDigite o número do defeito: ")

        try:
            escolha_defeito_int = int(escolha_defeito)
            if escolha_defeito_int == 0:
                print("\n\n----------------------------------------\n\n")
                exibir_menu_principal()  # Voltar ao menu principal
                break
            elif 1 <= escolha_defeito_int <= len(maquina['defeitos']):
                print("\nPor favor aguarde, Gemini Maint AI está processando sua solicitação...")
                time.sleep(1) # Delay para aguardar o print acima antes de iniciar a solicitação ao Gemini
                defeito_selecionado = maquina['defeitos'][escolha_defeito_int - 1]
                solicita_ajuda_ia(maquina, defeito_selecionado)
                break  # Sai do loop se a escolha for válida
            else:
                print("Escolha inválida. Por favor, digite um número válido.")
        except ValueError:
            print("Escolha inválida. Por favor, digite um número.")

In [12]:
#Solicita ajuda da IA:
def solicita_ajuda_ia(maquina, defeito_selecionado):
  prompt = f"""
  {system_instructions}
  Máquina: {maquina['nome']}
  Modelo: {maquina['modelo']}
  Fabricante: {maquina['fabricante']}
  Manual: {maquina['manual']}
  Defeito: {defeito_selecionado['descricao']}
  """

  response = model.generate_content(prompt)

  exibir_ajuda(maquina, defeito_selecionado, response)

In [13]:
#Função exibir ajuda:
def exibir_ajuda(maquina, defeito_selecionado, response):
  clear()
  print("\n\n----------------------------------------\n\n")
  print("*** AJUDA GEMINI MAINT AI ***")
  print(f"\nMáquina: {maquina['nome']}")
  print(f"Modelo: {maquina['modelo']}")
  print(f"Fabricante: {maquina['fabricante']}")
  print(f"\nDefeito: {defeito_selecionado['descricao']}\n")

  #print(prompt)
  print(response.text)

  while True:

    voltar_menu = input("\nDigite 0 para voltar ao menu principal: ")

    try:
        voltar_menu_int = int(voltar_menu)
        if voltar_menu_int == 0:
            print("\n\n----------------------------------------\n\n")
            exibir_menu_principal()  # Voltar ao menu principal
            break
        else:
            print("Escolha inválida. Por favor, digite um número válido.")
    except ValueError:
        print("Escolha inválida. Por favor, digite um número.")

In [ ]:
#Inicia o sistema
exibir_menu_principal()